# GeoJSON Analysis Notebook

This notebook reads and analyzes GeoJSON output files from the P1812 batch processor.

## Workflow
1. Load all `points_*.geojson` files from `data/output/geojson/`
2. Merge all GeoDataFrames
3. Classify electric field strength (Ep) into ranges
4. Save merged results with classifications

## Requirements
- geopandas
- pandas
- numpy


In [43]:
import geopandas as gpd
import pandas as pd 
from pathlib import Path
import numpy as np

In [ ]:
# Set up data paths relative to notebook location
from pathlib import Path

# Get the notebook's directory and navigate to project root
notebook_dir = Path.cwd()
project_root = Path.cwd() if (Path.cwd() / 'data').exists() else Path.cwd().parent
data_dir = project_root / 'data' / 'output' / 'geojson'

print(f"Notebook directory: {notebook_dir}")
print(f"Project root: {project_root}")
print(f"Data directory: {data_dir}")
print(f"Data directory exists: {data_dir.exists()}")


In [ ]:
# Load all GeoJSON point files
files = sorted(list(data_dir.glob('points_*.geojson')))

if not files:
    print(f"⚠️  No point files found in {data_dir}")
    print("Run batch processor first: python scripts/run_batch_processor.py")
else:
    print(f"✅ Found {len(files)} point files")
    for f in files[:3]:
        print(f"   - {f.name}")
    if len(files) > 3:
        print(f"   ... and {len(files) - 3} more")


In [44]:
folder = Path("/Users/aleksandra/Desktop/ITU/Mobile Simulation Tool/geojson")

files = list(folder.glob("points*.geojson"))

print(f"Found {len(files)} point files")

Found 11 point files


In [45]:
# Read and merge all GeoDataFrames
gdfs = []

for f in files:
    print(f"Loading {f.name}...")
    gdf = gpd.read_file(f)
    gdfs.append(gdf)

print(f"✅ Loaded {len(gdfs)} files")


Loaded all files


In [46]:
# Merge all GeoDataFrames
merged_gdf = gpd.GeoDataFrame(
    pd.concat(gdfs, ignore_index=True),
    crs=gdfs[0].crs
)

print(f"✅ Merged into single GeoDataFrame: {merged_gdf.shape}")

# Classify electric field strength into ranges
ep = merged_gdf["Ep"]

# Define bin edges (every 10 units)
bin_edges = np.arange(np.floor(ep.min()/10)*10,
                      np.ceil(ep.max()/10)*10 + 10,
                      10)

labels = [f\"{int(bin_edges[i])}-{int(bin_edges[i+1])}\"
          for i in range(len(bin_edges)-1)]

merged_gdf[\"Ep_class\"] = pd.cut(
    merged_gdf[\"Ep\"],
    bins=bin_edges,
    labels=labels,
    include_lowest=True
)

# Save merged results
output_file = data_dir / 'merged_points_Ep_ranges.geojson'
merged_gdf.to_file(output_file, driver=\"GeoJSON\")
print(f"✅ Saved merged results to {output_file}")

# Display summary statistics
print(\"\nField Strength Classification Summary:")
print(merged_gdf[[\"Ep\", \"Ep_class\"]].dropna().head())
print(\"\nEp class distribution:")
print(merged_gdf[\"Ep_class\"].value_counts().sort_index())
print(f\"\nBin edges: {bin_edges}")
print(f\"Min Ep: {ep.min():.2f} dBμV/m")
print(f\"Max Ep: {ep.max():.2f} dBμV/m")
print(f\"Total points: {merged_gdf.shape[0]}")


          Ep Ep_class
1  62.626625    60-70
2  80.847899    80-90
3  64.959749    60-70
4  54.059425    50-60
5  55.065352    50-60
Ep_class
80-90      92
60-70      82
90-100     82
70-80      72
50-60      49
100-110    19
Name: count, dtype: int64
Bins: [ 50.  60.  70.  80.  90. 100. 110.]
Min Ep: 50.549189509212596
Max Ep: 106.95000536454079
(407, 12)
          name          Lb         Ep  frequency   hrg  distance   htg  \
0  Transmitter         NaN        NaN        0.9   NaN       NaN  57.0   
1   Receiver_1  135.818225  62.626625        NaN  10.0      10.0   NaN   
2   Receiver_2  117.596951  80.847899        NaN  10.0      10.0   NaN   
3   Receiver_3  133.485101  64.959749        NaN  10.0      10.0   NaN   
4   Receiver_4  144.385425  54.059425        NaN  10.0      10.0   NaN   

   polarization                   geometry  lon  lat Ep_class  
0           1.0    POINT (-13.40694 9.345)  NaN  NaN      NaN  
1           NaN  POINT (-13.40653 9.43541)  NaN  NaN    60-70  
2    